Import Libraries


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

#tensorflow.keras.optimizers.legacy.Adam  --from tensorflow.keras.optimizers import Adam

Defining initial parameters

In [2]:
epochs = 100
learning_rate = 1e-3
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []

load image files from the dataset

In [3]:
image_files = [f for f in glob.glob(r'D:\Gender-Detection\gender_dataset_face' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

converting images to arrays and labelling the categories

In [4]:
for img in image_files:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2] # C:\Files\gender_dataset_face\woman\face_1162.jpg
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label]) # [[1], [0], [0], ...]

pre-processing

In [5]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)


split dataset for training and validation 

In [6]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=2) # [[1, 0], [0, 1], [0, 1], ...]
testY = to_categorical(testY, num_classes=2)

# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

Building model

In [8]:

def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

# compile the model
opt = Adam(learning_rate=learning_rate, decay=learning_rate/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])





train the model

In [9]:
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)



C:\Users\DELL\AppData\Local\Temp\ipykernel_8308\1007086755.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),


Epoch 1/100
28/28 [==============================] - 17s 548ms/step - loss: 0.7897 - accuracy: 0.7243 - val_loss: 0.8085 - val_accuracy: 0.5281
Epoch 2/100
28/28 [==============================] - 15s 542ms/step - loss: 0.4820 - accuracy: 0.8214 - val_loss: 0.8518 - val_accuracy: 0.5281
Epoch 3/100
28/28 [==============================] - 15s 526ms/step - loss: 0.3854 - accuracy: 0.8560 - val_loss: 0.7201 - val_accuracy: 0.5281
Epoch 4/100
28/28 [==============================] - 15s 536ms/step - loss: 0.3367 - accuracy: 0.8759 - val_loss: 0.7367 - val_accuracy: 0.5433
Epoch 5/100
28/28 [==============================] - 15s 527ms/step - loss: 0.3203 - accuracy: 0.8798 - val_loss: 0.6660 - val_accuracy: 0.5714
Epoch 6/100
28/28 [==============================] - 15s 530ms/step - loss: 0.3104 - accuracy: 0.8815 - val_loss: 1.6324 - val_accuracy: 0.5281
Epoch 7/100
28/28 [==============================] - 15s 529ms/step - loss: 0.2766 - accuracy: 0.8939 - val_loss: 1.2048 - val_accuracy:

28/28 [==============================] - 15s 527ms/step - loss: 0.0563 - accuracy: 0.9798 - val_loss: 0.2013 - val_accuracy: 0.9394
Epoch 58/100
28/28 [==============================] - 15s 529ms/step - loss: 0.0485 - accuracy: 0.9792 - val_loss: 0.0851 - val_accuracy: 0.9762
Epoch 59/100
28/28 [==============================] - 15s 526ms/step - loss: 0.0509 - accuracy: 0.9803 - val_loss: 0.7459 - val_accuracy: 0.8182
Epoch 60/100
28/28 [==============================] - 15s 528ms/step - loss: 0.0615 - accuracy: 0.9781 - val_loss: 0.1086 - val_accuracy: 0.9632
Epoch 61/100
28/28 [==============================] - 15s 534ms/step - loss: 0.0551 - accuracy: 0.9820 - val_loss: 0.1534 - val_accuracy: 0.9545
Epoch 62/100
28/28 [==============================] - 15s 529ms/step - loss: 0.0526 - accuracy: 0.9815 - val_loss: 0.1149 - val_accuracy: 0.9610
Epoch 63/100
28/28 [==============================] - 15s 526ms/step - loss: 0.0423 - accuracy: 0.9848 - val_loss: 0.1229 - val_accuracy: 0.958

save the model to disk

In [10]:
model.save('gender_detection.model')

INFO:tensorflow:Assets written to: gender_detection.model\assets


INFO:tensorflow:Assets written to: gender_detection.model\assets


In [ ]:
# plot training/validation loss/accuracy
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0,N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0,N), H.history["val_acc"], label="val_acc")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")

# save plot to disk
plt.savefig('plot.png')